In [1]:
# !pip install transformers

In [2]:
# cd /content/drive/MyDrive/commit_folder/competition/tour_ai

In [9]:
import os
os.getcwd()

'/root/share/tour_ai/code/model'

In [22]:
os.chdir('/root/share/')

In [31]:
server = 'docker'
from tour_ai.code.utils import dataset, models

#
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [32]:
if server == 'colab':
  mecab_data_path = 'data/mecab_data.csv'
elif server == 'docker':
  mecab_data_path = '/tour_ai/data/mecab_data.csv'

In [29]:
from sklearn import preprocessing
from transformers import AutoTokenizer

def data_preprocess(data_path, infer_yn = False):
  
  data = pd.read_csv(data_path)
  data = data[['img_path','cat3','mecab_data']]
  data.columns = ['img_path','label','sentence']

  label_encoding = preprocessing.LabelEncoder()
  label_encoding.fit(data['label'].values)
  data['label'] = label_encoding.transform(data['label'].values)
  
  return data, label_encoding

def tokenize_sentence(data, tokenizer_name = 'klue/roberta-small', padding = 'max_length', max_length = 300, truncation = True):
  if tokenizer_name == 'klue/roberta-small':
    tokenizer = AutoTokenizer.from_pretrained('klue/roberta-small')

  data['sentence'] = data['sentence'].apply(lambda x: tokenizer(x, padding=padding, max_length=max_length, truncation=True, return_tensors='pt'))
  return data

In [30]:
data, label_encoding = data_preprocess(mecab_data_path)
data = tokenize_sentence(data)

# train_test_split
train, valid = train_test_split(data, test_size=0.05, random_state=42)

FileNotFoundError: [Errno 2] No such file or directory: 'data/mecab_data.csv'

In [13]:
train_dataset = dataset.TextDataset(train)
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle=True)

valid_dataset = dataset.TextDataset(valid)
valid_dataloader = DataLoader(valid_dataset, batch_size = 32, shuffle=True)

In [14]:
model = models.TextModel(num_classes=len(label_encoding.classes_), text_model ='bert_klue')

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f